# Before you start

- You'll need a Google Drive account to connect to.
- Ensure you've selected the `GPU` runtime type in Colab, under `Runtime->Change runtime type->Hardware acceleration`

# Mount Google Drive

Google Drive is typically the easiest way to persist files across notebooks and sessions.

In [1]:
# Mount Google Drive (will prompt for permission)
%cd /content/
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

/content
Mounted at /content/gdrive


# Build wheelhouse

Setup is painfully slow for Colab notebooks, these scripts try to pre-build wheels to speed things up.

You should be able to build once and re-use. If you find you're getting errors with `tiny-cuda-nn`, you might need to rebuild - although this should be rare (e.g. if Colab updates their python runtime).

You can either use the nerfstudio version we build here, or build your own from source in your own notebook - so long as the dependencies remain the same.

In [2]:
%cd /content/
!pip install --upgrade pip

# Create wheelhouse that we'll populate with pre-built wheels
!mkdir -p /content/gdrive/MyDrive/colab/wheelhouse

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [ ]:
# Build the python bindings from tiny-cuda-nn and copy to wheelhouse
%cd /content/
!git clone --recursive https://github.com/NVlabs/tiny-cuda-nn
%cd tiny-cuda-nn/bindings/torch
!python setup.py bdist_wheel
%cp dist/*.whl /content/gdrive/MyDrive/colab/wheelhouse/

# Test installing tinycudann from wheelhouse
%cd /content/
!pip install --no-index --find-links=/content/gdrive/MyDrive/colab/wheelhouse tinycudann


/content
Cloning into 'tiny-cuda-nn'...
remote: Enumerating objects: 3773, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3773 (delta 172), reused 248 (delta 150), pack-reused 3454
Receiving objects: 100% (3773/3773), 19.65 MiB | 40.99 MiB/s, done.
Resolving deltas: 100% (2364/2364), done.
Submodule 'dependencies/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'dependencies/cutlass'
Submodule 'dependencies/fmt' (https://github.com/fmtlib/fmt) registered for path 'dependencies/fmt'
Cloning into '/content/tiny-cuda-nn/dependencies/cutlass'...
remote: Enumerating objects: 19806, done.        
remote: Counting objects: 100% (2454/2454), done.        
remote: Compressing objects: 100% (991/991), done.        
remote: Total 19806 (delta 1589), reused 2209 (delta 1427), pack-reused 17352        
Receiving objects: 100% (19806/19806), 30.97 MiB | 24.95 MiB/s, done.
Resolving deltas: 100% (14288/14288), don

In [ ]:
# Build nerfstudio into wheelhouse
# Substitute a different branch/commit, if required
%cd /content/
!git clone -b nerfplayer-testing --single-branch https://github.com/chris838/nerfstudio.git
%cd nerfstudio
!pip wheel . --wheel-dir /content/gdrive/MyDrive/colab/wheelhouse

# Test installing nerfstudio from wheelhouse
%cd /content/
!pip install --no-index --find-links=/content/gdrive/MyDrive/colab/wheelhouse nerfstudio

# Build COLMAP

You can skip this step if you're not using COLMAP, e.g. you already have pre-processed images with camera poses, or you're using Polycam output.

If you want to train nerfstudio from raw images or video, you will need COLMAP installed.

Building COLMAP takes a while. We build here once and store to Google Drive, running it later is then very fast. 

By building ourselves, we can use colmap wihtout installing `conda` - saving more time and avoiding a kernel restart.

In [ ]:
# Dependencies
!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev
  
# Build
%cd /content/
!git clone https://github.com/colmap/colmap
%cd colmap
%mkdir build
%cd build
!cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES='70'
!ninja
!sudo ninja install

# Backup build outputs to GDrive
%cd /content/
%mkdir -p /content/gdrive/MyDrive/colab/colmap/local/bin
%mkdir -p /content/gdrive/MyDrive/colab/colmap/local/lib
%mkdir -p /content/gdrive/MyDrive/colab/colmap/local/share/applications
%mkdir -p /content/gdrive/MyDrive/colab/colmap/local/include

%cp /usr/local/bin/colmap /content/gdrive/MyDrive/colab/colmap/local/bin/
%cp -r /usr/local/lib/colmap /content/gdrive/MyDrive/colab/colmap/local/lib/
%cp /usr/local/share/applications/COLMAP.desktop /content/gdrive/MyDrive/colab/colmap/local/share/applications/
%cp -r /usr/local/share/colmap/ /content/gdrive/MyDrive/colab/colmap/local/share/
%cp -r /usr/local/include/colmap /content/gdrive/MyDrive/colab/colmap/local/include/

%cd /content/gdrive/MyDrive/colab/colmap
!zip -r local.zip ./local && rm -rf ./local

# [OPTIONAL] Test nerfstudio install from Google Drive

Redundant if you're running through this end-to-end, but useful for debugging.

In [ ]:
%cd /content/
!pip install --no-index --find-links=/content/gdrive/MyDrive/colab/wheelhouse nerfstudio tinycudann

# Copy test data to Google Drive

Copy test datasets to Google Drive, which we can then test in the main notebook.

This may seem superfluous, but it does provide a blueprint for putting your own data in Google Drive and accessing it from Colab notebooks.

In [ ]:
# Create data dir
%cd /content/
%mkdir /content/gdrive/MyDrive/colab/data

In [ ]:
# Copy nerf_synthetic dataset
%cd /content/
!ns-download-data blender --save-dir /content/gdrive/MyDrive/colab/data

In [ ]:
# Copy dynamic nerf dataset
!ns-download-data dnerf --save-dir /content/gdrive/MyDrive/colab/data